In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.10:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.7/dist-packages/kaggle-1.5.10.dist-info/*
    /usr/local/lib/python3.7/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 5.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=808e6f69d6ba6490e17e09fb53ae3bdee09626fdbee06a8ca481a3f7c6a5dc30
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
-rw-r--r-- 1 root root 65 Mar 17 06:25 kaggle.json


In [2]:
!kaggle competitions download -c sejong-ai-challenge-p2

 57% 17.0M/29.8M [00:00<00:01, 12.0MB/s]
100% 29.8M/29.8M [00:01<00:00, 31.0MB/s]


In [3]:
!unzip sejong-ai-challenge-p2.zip

Archive:  sejong-ai-challenge-p2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [16]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

train = shuffle(train, random_state=0)

train

,Unnamed: 0,Text,Label
71019,64872,Good product. It has a slightly plastic feel ...,1
151264,207111,it worked on my phone but it didn't come with ...,0
126784,77813,We use this for my teenage daughters phone and...,1
98978,200321,Great !!!,1
58578,148107,I love collecting phone cases because I like t...,1
...,...,...,...
152315,260160,Did not like.,0
176963,49409,"I like the overall design of this case a lot, ...",0
117952,10677,Breaks easily. Mine didn't even last a week,0
173685,167021,I had the phone for a month and absolutely lov...,0


In [13]:
print(train.shape)
print(train['Text'][0])

(194140, 3)
Great price and quick service, the product is a back case with no front protection, but alas thats why I ordered the sleeves for the front. Seems very durable and color is very vibrant.


In [18]:
!pip install fasttext

In [19]:
train_df = train.iloc[:133000, :]
val_df = train.iloc[133000: , :]

In [20]:
train_df.head(10)

,Unnamed: 0,Text,Label
71019,64872,Good product. It has a slightly plastic feel ...,1
151264,207111,it worked on my phone but it didn't come with ...,0
126784,77813,We use this for my teenage daughters phone and...,1
98978,200321,Great !!!,1
58578,148107,I love collecting phone cases because I like t...,1
114291,37080,Great product!,1
120606,107017,Really improved the operation of my little Bao...,1
138334,11578,I am very happy with this band. It doesn't ru...,1
143363,5853,Excellent!,1
192050,48868,Awesome product!!!,1


In [27]:
import fasttext

file = open('fasttexttrain.txt', 'w+')
for i in train_df.index:
    line = '__label__' + str(train['Label'][i]) + ' ' + str(train['Text'][i])
    file.write(line+'\n')

In [22]:
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=15, wordNgrams=3, minCount=2, maxn=10, verbose=0)

In [24]:
result = text_clf_model.predict('Great price and quick service, the product is a back case with no front protection, but alas thats why I ordered the sleeves for the front. Seems very durable and color is very vibrant.', k=1)
print(result)

(('__label__1',), array([0.91550618]))


In [31]:
val_df['Text'] = val_df['Text'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
for i in val_df.index:
    val_df['Text'][i] =val_df['Text'][i].replace('\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
val_train = val_df.drop(['Label'], axis=1)
val_label = val_df['Label']

In [34]:
from sklearn.metrics import f1_score

val_pred = [text_clf_model.predict(x)[0][0].split('__')[-1] for x in val_train['Text']]

print('Val_f1_Score: ', f1_score(val_label.astype(str), val_pred, average='macro'))

Val_f1_Score:  0.9129176740076963


In [ ]:
test['Text'].loc[7]

"See if you sound like me: - Large family with multiple devices that need charging either by Apple cables or Micro USB - Cables are always getting lost, eaten, frayed/broken, lent, or sucked into another dimensionI'm usually on Ebay or Amazon once every 3 months to buy a wholesale pack of micro USB or Apple cables because the 9+ devices we charge at home go through cables like most families go through milk.  Most cables I buy do not stand up to our abuse, and the usual break point is the cable to micro USB head.These cables from Pro Rich seem to be different.  The break point looks to be more reinforced, the cables themselves have some heft and sturdiness to them, and the LENGTH(!) is outstanding.  I would recommend these to anyone who feels like a wholesale dealer of charging cables (or anyone that seems to go through them on a regular basis)."

In [35]:
for i in test.index:
    test['Text'][i] =test['Text'][i].replace('\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:

for i in test.index:
    test_pred = text_clf_model.predict(test['Text'][i], k=1)[0][0]
    submission['Label'][i] =  test_pred.split('__')[-1]


print(submission)

          id  Label
0          0      1
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
83199  83199      0
83200  83200      0
83201  83201      0
83202  83202      0
83203  83203      1

[83204 rows x 2 columns]


In [37]:
submission.to_csv('submission1.csv', index=None, header=True)

In [38]:
!kaggle competitions submit -c sejong-ai-challenge-p2 -f submission1.csv -m "Message"

100% 639k/639k [00:08<00:00, 77.5kB/s]
Successfully submitted to Sejong AI Challenge 문제2

In [ ]:
## 0.91398

## shuffle + wordngram = 3 + validation set 생성 + fasttext